# C3.5

In [2]:
tag = 'EURUSD'

In [3]:
import pandas as pd
import numpy as np

# 读取CSV文件
news_scores = pd.read_csv(f'{tag}_news_scores.csv')
price_data = pd.read_csv(f'{tag}.csv')

# 将Created Date转换为datetime类型，并去除时区信息
news_scores['Created Date'] = pd.to_datetime(news_scores['Created Date']).dt.tz_localize(None)
news_scores['Created Date'] = news_scores['Created Date'].dt.floor('T')  # 将时间向下取整到分钟

# 将datetime转换为datetime类型
price_data['datetime'] = pd.to_datetime(price_data['datetime'])
# 将price_data设置为以datetime为索引
price_data.set_index('datetime', inplace=True)


# Convert price columns to float
price_data['close'] = pd.to_numeric(price_data['close'], errors='coerce')
price_data['open'] = pd.to_numeric(price_data['open'], errors='coerce')
price_data['high'] = pd.to_numeric(price_data['high'], errors='coerce')
price_data['low'] = pd.to_numeric(price_data['low'], errors='coerce')



# 定义计算收益的函数
def calculate_returns(row, n, price_data):
    current_time = row['Created Date']
    future_time = current_time + pd.Timedelta(minutes=n)
    
    try:
        current_close = price_data.loc[current_time, 'close']
        current_open = price_data.loc[current_time, 'open']
        future_close = price_data.loc[future_time, 'close']
        future_open = price_data.loc[future_time, 'open']
        
        r_close = (future_close - current_close) / current_close
        r_open = (future_open - current_open) / current_open
        
        return pd.Series({'r_close_{}'.format(n): r_close, 'r_open_{}'.format(n): r_open})
    except KeyError:
        return pd.Series({'r_close_{}'.format(n): np.nan, 'r_open_{}'.format(n): np.nan})

# 定义要计算的时间间隔
time_intervals = [1, 3, 5, 10]

# 计算不同时间间隔的收益
for n in time_intervals:
    news_scores = news_scores.join(news_scores.apply(lambda row: calculate_returns(row, n, price_data), axis=1))

# 保存结果到新的CSV文件
news_scores.to_csv(f'{tag}_scores_returns.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'EURUSD_news_scores.csv'

# G4o

In [3]:
import pandas as pd

# Load the data
news_scores_df = pd.read_csv(f'{tag}_news_scores.csv')
price_data_df = pd.read_csv(f'{tag}.csv')

# Convert date columns to datetime
news_scores_df['Created Date'] = pd.to_datetime(news_scores_df['Created Date'])
price_data_df['datetime'] = pd.to_datetime(price_data_df['datetime'])

# Remove timezone information for consistency
news_scores_df['Created Minute'] = news_scores_df['Created Date'].dt.tz_localize(None).dt.floor('T')
price_data_df['datetime'] = price_data_df['datetime'].dt.tz_localize(None)

# Convert price columns to float
price_data_df['close'] = pd.to_numeric(price_data_df['close'], errors='coerce')
price_data_df['open'] = pd.to_numeric(price_data_df['open'], errors='coerce')
price_data_df['high'] = pd.to_numeric(price_data_df['high'], errors='coerce')
price_data_df['low'] = pd.to_numeric(price_data_df['low'], errors='coerce')

# Sort the price data by datetime
price_data_df = price_data_df.sort_values('datetime')

# List of n values for which we want to calculate returns
n_values = [1, 3, 5, 10]

# Define a function to calculate returns
def calculate_returns(row):
    results = {}
    created_minute = row['Created Minute']
    
    # Find the price data row for the created minute
    price_row = price_data_df.loc[price_data_df['datetime'] == created_minute]
    if price_row.empty:
        for n in n_values:
            results[f'r_close_{n}'] = None
            results[f'r_open_{n}'] = None
        return results

    close_price1 = price_row['close'].values[0]
    open_price1 = price_row['open'].values[0]

    for n in n_values:
        future_minute = created_minute + pd.Timedelta(minutes=n)
        future_price_row = price_data_df[price_data_df['datetime'] == future_minute]
        
        if not future_price_row.empty:
            close_price2 = future_price_row['close'].values[0]
            open_price2 = future_price_row['open'].values[0]
            results[f'r_close_{n}'] = (close_price2 - close_price1) / close_price1
            results[f'r_open_{n}'] = (open_price2 - open_price1) / open_price1
        else:
            results[f'r_close_{n}'] = None
            results[f'r_open_{n}'] = None

    return results

# Apply the function to calculate returns
returns = news_scores_df.apply(calculate_returns, axis=1, result_type='expand')
news_scores_df = pd.concat([news_scores_df, returns], axis=1)

# Save the results to a new CSV file
news_scores_df.to_csv(f'{tag}_scores_returns_gpt4o.csv', index=False)

/var/folders/08/51ch_vwn57lgdw3v47gn9rjh0000gn/T/ipykernel_49531/212994545.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  news_scores_df['Created Minute'] = news_scores_df['Created Date'].dt.tz_localize(None).dt.floor('T')


In [4]:
df1 = news_scores.dropna()
df2 = news_scores_df.dropna()
df1 = df1[['r_close_1',
       'r_open_1', 'r_close_3', 'r_open_3', 'r_close_5', 'r_open_5',
       'r_close_10', 'r_open_10']]
df2 = df2[['r_close_1',
       'r_open_1', 'r_close_3', 'r_open_3', 'r_close_5', 'r_open_5',
       'r_close_10', 'r_open_10']]
# 检查两个DataFrame的同名列是否完全相同
if df1.equals(df2):
    print("两个DataFrame完全相同")
else:
    # 找到不同的部分
    diff = df1 != df2
    diff_df = pd.concat([df1[diff], df2[diff]], keys=['df1', 'df2'])
    print("不同的部分：")
    print(diff_df)

两个DataFrame完全相同


In [1]:
news_scores.columns

NameError: name 'news_scores' is not defined

In [6]:
import pandas as pd
import numpy as np
df = news_scores.copy()
df
df = df.rename(columns={'A': 'B'})

# 假设您的 DataFrame 名为 df
# 需要处理的列名列表
columns_to_process = ['r_close_1', 'r_open_1', 'r_close_3', 'r_open_3', 'r_close_5', 'r_open_5', 'r_close_10', 'r_open_10']

for col in columns_to_process:
    new_col = f"{col}_v"
    
    # 创建新列，初始值与原列相同
    df[new_col] = df[col]
    
    # 对 Vader_sentiment_title 小于 0 的行，新列的值乘以 -1
    mask = df['Vader_sentiment_title'] < 0
    df.loc[mask, new_col] = df.loc[mask, col] * -1

# 验证结果
df

,Created Date,Title,Tag,FinBERT_sentiment_title,ABSA_Bert_sentiment_title_EUR,Vader_sentiment_title,r_close_1,r_open_1,r_close_3,r_open_3,...,r_close_10,r_open_10,r_close_1_v,r_open_1_v,r_close_3_v,r_open_3_v,r_close_5_v,r_open_5_v,r_close_10_v,r_open_10_v
0,2024-04-29 00:50:00,"EUR/USD holds positive ground above 1.0700, ey...",EURUSD,0.794606,0.556653,0.5574,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-29 06:13:00,EUR/USD Price Analysis: Keeps steady above 1.0...,EURUSD,0.999999,0.000000,0.0000,0.0,0.000000,0.000093,0.000000,...,0.000187,0.000187,0.0,0.000000,0.000093,0.000000,0.000093,0.000093,0.000187,0.000187
2,2024-04-29 14:33:00,"EUR/USD retreats ahead of Eurozone, US data-pa...",EURUSD,0.000000,-0.694304,0.0000,0.0,-0.000187,0.000000,-0.000093,...,-0.000093,-0.000093,0.0,-0.000187,0.000000,-0.000093,0.000000,0.000000,-0.000093,-0.000093
3,2024-04-29 22:28:00,EUR/USD finds support near 1.0720 after slow g...,EURUSD,0.000000,0.000000,0.4019,0.0,0.000000,-0.000187,-0.000187,...,0.000000,0.000093,0.0,0.000000,-0.000187,-0.000187,0.000000,-0.000093,0.000000,0.000093
4,2024-04-30 04:31:00,EUR/USD Price Analysis: Manages to hold above ...,EURUSD,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,...,-0.000093,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000093,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2024-08-26 12:55:00,EUR/USD: Consolidates under 1.12 for a break a...,EURUSD,0.000000,0.000000,0.0000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,2024-08-26 13:10:00,EUR/USD: Bias to fade rally – OCBC,EURUSD,0.000000,0.000000,-0.1027,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,2024-08-26 14:21:00,"EUR/USD corrects from 1.1200 as Fed, ECB Septe...",EURUSD,0.000000,0.000000,-0.2732,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,2024-08-26 23:13:00,EUR/USD backslides in broad-market Greenback b...,EURUSD,0.000000,0.000000,0.0000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
from datetime import timedelta

def calculate_cumulative_returns(df, n):
    df = df.sort_values('createDate')
    df['Cum_r_n'] = 0.0
    last_trade_date = df.iloc[0]['createDate']
    cum_return = 1.0

    for i, row in df.iterrows():
        if i == 0 or (row['createDate'] - last_trade_date) >= timedelta(minutes=n):
            cum_return *= (1 + row['r_n'])
            last_trade_date = row['createDate']
        
        df.at[i, 'Cum_r_n'] = cum_return - 1

    return df

# 使用示例
# df = pd.read_csv('your_data.csv', parse_dates=['createDate'])
# n = 5  # 假设n为5分钟
# result = calculate_cumulative_returns(df, n)
# print(result)

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def calculate_metrics(returns):
    total_return = returns.iloc[-1]
    daily_returns = (1 + returns).resample('D').prod() - 1
    
    # Calculate max drawdown
    cum_returns = (1 + returns).cumprod()
    running_max = np.maximum.accumulate(cum_returns)
    drawdown = (cum_returns - running_max) / running_max
    max_drawdown = drawdown.min()
    
    # Calculate Sharpe ratio (assuming risk-free rate of 0)
    sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std()
    
    return {
        'Total Return': total_return,
        'Max Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe_ratio
    }

def calculate_additional_metrics(df, column):
    positive_returns = df[df[column] > 0][column]
    negative_returns = df[df[column] < 0][column]
    
    win_ratio = len(positive_returns) / len(df[column])
    pnl_ratio = positive_returns.mean() / abs(negative_returns.mean()) if len(negative_returns) > 0 else np.inf
    risk_return_ratio = (win_ratio / (1 - win_ratio)) * pnl_ratio if win_ratio < 1 else np.inf
    
    return {
        'Win Ratio': win_ratio,
        'P&L Ratio': pnl_ratio,
        'Risk Return Ratio': risk_return_ratio
    }

def plot_returns_and_metrics(df):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces for each return series
    for col in ['Cum_r_5', 'Cum_r_30', 'Cum_r_60']:
        fig.add_trace(
            go.Scatter(x=df['createDate'], y=df[col], name=col),
            secondary_y=False,
        )

    # Update layout
    fig.update_layout(
        title='Cumulative Returns',
        xaxis_title='Time',
        yaxis_title='Return',
        legend_title='Return Type',
        hovermode="x unified",
        template="plotly_white",
    )

    # Show the plot
    fig.show()

    # Calculate and print metrics
    for col in ['Cum_r_5', 'Cum_r_30', 'Cum_r_60']:
        cum_metrics = calculate_metrics(df.set_index('createDate')[col])
        print(f"\nMetrics for {col}:")
        for metric, value in cum_metrics.items():
            print(f"{metric}: {value:.4f}")

    # Calculate and print additional metrics for r_n_v columns
    r_columns = [col for col in df.columns if col.startswith('r_') and col.endswith('_v')]
    for col in r_columns:
        add_metrics = calculate_additional_metrics(df, col)
        print(f"\nAdditional Metrics for {col}:")
        for metric, value in add_metrics.items():
            print(f"{metric}: {value:.4f}")

# Assuming df is your DataFrame with 'createDate', 'Cum_r_5', 'Cum_r_30', 'Cum_r_60', 'r_1_v', 'r_3_v', etc. columns
df['createDate'] = pd.to_datetime(df['createDate'])
plot_returns_and_metrics(df)